This notebook exists to investigate the need of the parameters included in the dataset. Each parameter is tested against the others to see if they are statistically correlated....
Then wrapper methods are used to determine which ones are most important....
Want to reduce number of parameters needed to produce a good output so to reduce the chance of overfitting and increase the generalisation of the model.

In [7]:
%run "Parameter_Estimation.ipynb" #allowing access to parameters

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:09<00:00, 21.36it/s]


In [161]:
from scipy.stats import pearsonr #for comparisson of correlations
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, mutual_info_classif #for forward feature selection
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [217]:
def get_balanced_accuracy(y_test, y_pred):
    """
    balanced accuracy....
    """
    num_healthy_true = np.sum([x=='Healthy' for x in y_test])
    num_unhealthy_true = len(y_test) - num_healthy_true
    count_healthy_accurate = 0
    count_unhealthy_accurate = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            count_unhealthy_accurate +=1
        elif y_pred[i] == y_test[i] == 'Healthy':
            count_healthy_accurate +=1
    healthy_percentage = count_healthy_accurate/num_healthy_true
    unhealthy_percentage = count_unhealthy_accurate/num_unhealthy_true
    balanced_accuracy = (healthy_percentage + unhealthy_percentage) * 0.5
    return balanced_accuracy

def get_specificity(y_test, y_pred):
    """
    true negative rate
    """
    true_negative = 0
    false_positive = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Healthy':
            true_negative += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
    
    return true_negative / (true_negative+false_positive)
            
def get_f1_score(y_test, y_pred):
    """
    balance between precision and recall
    """
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            true_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Unhealthy':
            false_negative += 1
    return (2*true_positive)/(2*true_positive+false_positive+false_negative)

def scoring_function(model, X, y):
    """
    change to incorporate balanced accuracy 
    """
    
    y_pred = model.predict(X)
    y_test = y
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    #specificity = get_specificity(y_test, y_pred)
    f1 = get_f1_score(y_test, y_pred)
    
    
    return f1*0.7 + balanced_acc*0.3

def convert_multi_dict_to_array(params):
    no_features = len(params)
    params_array = np.zeros((no_channels, no_patients, no_features))
    
    for j in range(0, no_channels):
        for i, values in enumerate(params.values()):
            params_array[j][:, i] = values[j]
    return params_array, no_features

def convert_single_dict_to_array(params):
    no_features = len(params)
    params_array = np.zeros((no_patients, no_features))
    for i, values in enumerate(params.values()):
        params_array[:, i] = values
    return params_array, no_features

def reduce_parameter_set(params, selected_indices, channel):
    select_params = {}
    for i, key in enumerate(params.keys()):
        if i in selected_indices:
            select_params[key] = params[key][channel]
    return select_params
        
def reduce_parameter_set_single(params, selected_indices):
    select_params = {}
    for i, key in enumerate(params.keys()):
        if i in selected_indices:
            select_params[key] = params[key]
    return select_params
    

First convert the parameters to an array for easy investigation.

In [205]:
params_array, no_features = convert_multi_dict_to_array(params)

To select the most important features to be used in analysis used forward sfs

In [206]:
def forward_selection(params, scorer=scoring_function, k=6):
    
    X_train, X_test, y_train, y_test = train_test_split(params, health_state, test_size=0.3, stratify = health_state)

    #initialise svc
    svm = SVC(class_weight='balanced')

    SFS_forward = SequentialFeatureSelector(estimator=svm, scoring=scoring_function, cv=3, n_features_to_select=k)

    SFS_forward.fit(X_train, y_train)

    selected_indices = SFS_forward.get_support(indices=True)
    
    return selected_indices

Filter methods

In [207]:
def filter_method(params, health_state, k=6, scorer=mutual_info_classif):
    # Apply SelectKBest with mutual information
    selector = SelectKBest(score_func=scorer, k=k)
    
    X_new = selector.fit_transform(params, health_state)
    
    # Get the indices of selected features
    selected_indices = selector.get_support(indices=True)
    
    return selected_indices

Combining approaches

In [226]:
selected_params = {}
for i in range(0, no_channels):
    #find indices to keep through filter method
    chosen_indices_filt = filter_method(params_array[i], health_state, k=7)
    
    #reduce the parameter based on these indices
    reduced_params = reduce_parameter_set(params, chosen_indices_filt, i)

    #convert new params dict back into array
    reduced_params_array, no_features = convert_single_dict_to_array(reduced_params)

    #find 3 best features through sfs
    chosen_indices_filt_sfs = forward_selection(reduced_params_array, scorer=scoring_function, k=3)

    selected_params[i] = reduce_parameter_set_single(reduced_params, chosen_indices_filt_sfs)
    